In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.python.keras.layers import Dense, LSTM
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.callbacks import EarlyStopping

In [2]:
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('RSCCASN.csv')
df = pd.read_csv('RNNonTimesries\RSCCASN.csv', parse_dates=True, index_col='DATE')

In [ ]:
test_size = 18
test_ind = len(df) - test_size
train = df.iloc[:test_ind]
test = df.iloc[test_ind:]

scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)
print(train.shape)

In [ ]:
length = 12
batch_size = 1

generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=1)

In [ ]:
model = Sequential()
n_feature = 1

model.add(LSTM(750,activation='relu', input_shape=(length, n_feature)))
model.add(Dense(375, activation='relu'))
model.add(Dense(180, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

print(model.summary())

early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=2)

In [ ]:
validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size=1)

In [ ]:
hist = model.fit_generator(generator, epochs=20, validation_data=validation_generator, callbacks=[early_stop])

losses = pd.DataFrame(hist.history)
losses.plot()
losses.to_csv('RNNonTimeserie.csv', index=False)

model.save('RNNonTimeserie.h5')

In [ ]:
losses = pd.read_csv('RNNonTimeserie.csv')
later_model = load_model('RNNonTimeserie.h5')

losses.plot()

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_feature))

In [ ]:
first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_feature))

test_prediction = []

In [ ]:
for i in range(len(test)):
    current_pred = later_model.predict(current_batch)
    test_prediction.append(current_pred)
    current_batch = np.append(current_batch[:,1:,:], [current_pred], axis=1)

In [ ]:
test_prediction = scaler.inverse_transform(np.array(test_prediction).reshape(18, 1))
test['Prediction'] = test_prediction
print(test)

In [ ]:
test.plot()